In [58]:
import numpy as np
import random
import pandas as pd
import nltk
import re
import html
import transformers
from transformers import BertTokenizer
import torch

In [35]:
!pip install transformers

     |████████████████████████████████| 6.3 MB 217 kB/s eta 0:00:01
     |████████████████████████████████| 3.8 MB 395 kB/s eta 0:00:01


In [2]:
!pip install nltk

In [3]:
import datasets 
dataset = datasets.load_dataset('ucberkeley-dlab/measuring-hate-speech', 'binary')   
df = dataset['train'].to_pandas()
df.describe()

Using custom data configuration ucberkeley-dlab--measuring-hate-speech-c32713cabe528196
Found cached dataset parquet (/Users/rubenhayat/.cache/huggingface/datasets/ucberkeley-dlab___parquet/ucberkeley-dlab--measuring-hate-speech-c32713cabe528196/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/1 [00:00<?, ?it/s]

,comment_id,annotator_id,platform,sentiment,respect,insult,humiliate,status,dehumanize,violence,...,hatespeech,hate_speech_score,infitms,outfitms,annotator_severity,std_err,annotator_infitms,annotator_outfitms,hypothesis,annotator_age
count,135556.000000,135556.000000,135556.000000,135556.000000,135556.000000,135556.00000,135556.000000,135556.000000,135556.000000,135556.000000,...,135556.000000,135556.000000,135556.000000,135556.000000,135556.000000,135556.000000,135556.000000,135556.000000,135556.000000,135451.000000
mean,23530.416138,5567.097812,1.281352,2.954307,2.828875,2.56331,2.278638,2.698575,1.846211,1.052045,...,0.744733,-0.567428,1.034322,1.001052,-0.018817,0.300588,1.007158,1.011841,0.014589,37.910772
std,12387.194125,3230.508937,1.023542,1.231552,1.309548,1.38983,1.370876,0.898500,1.402372,1.345706,...,0.932260,2.380003,0.496867,0.791943,0.487261,0.236380,0.269876,0.675863,0.613006,11.641276
min,1.000000,1.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,...,0.000000,-8.340000,0.100000,0.070000,-1.820000,0.020000,0.390000,0.280000,-1.578693,18.000000
25%,18148.000000,2719.000000,0.000000,2.000000,2.000000,2.00000,1.000000,2.000000,1.000000,0.000000,...,0.000000,-2.330000,0.710000,0.560000,-0.380000,0.030000,0.810000,0.670000,-0.341008,29.000000
50%,20052.000000,5602.500000,1.000000,3.000000,3.000000,3.00000,3.000000,3.000000,2.000000,0.000000,...,0.000000,-0.340000,0.960000,0.830000,-0.020000,0.340000,0.970000,0.850000,0.110405,35.000000
75%,32038.250000,8363.000000,2.000000,4.000000,4.000000,4.00000,3.000000,3.000000,3.000000,2.000000,...,2.000000,1.410000,1.300000,1.220000,0.350000,0.420000,1.170000,1.130000,0.449555,45.000000
max,50070.000000,11142.000000,3.000000,4.000000,4.000000,4.00000,4.000000,4.000000,4.000000,4.000000,...,2.000000,6.300000,5.900000,9.000000,1.360000,1.900000,2.010000,9.000000,0.987511,81.000000


In [2]:
!pip install datasets

     |████████████████████████████████| 462 kB 8.0 MB/s eta 0:00:01
     |████████████████████████████████| 132 kB 29.3 MB/s eta 0:00:01
     |████████████████████████████████| 143 kB 20.6 MB/s eta 0:00:01
     |████████████████████████████████| 359 kB 50.9 MB/s eta 0:00:01
     |████████████████████████████████| 190 kB 33.7 MB/s eta 0:00:01
     |████████████████████████████████| 110 kB 46.5 MB/s eta 0:00:01
     |████████████████████████████████| 24.4 MB 43.0 MB/s eta 0:00:01
     |████████████████████████████████| 60 kB 27.1 MB/s eta 0:00:01


In [4]:
df

,comment_id,annotator_id,platform,sentiment,respect,insult,humiliate,status,dehumanize,violence,...,annotator_religion_hindu,annotator_religion_jewish,annotator_religion_mormon,annotator_religion_muslim,annotator_religion_nothing,annotator_religion_other,annotator_sexuality_bisexual,annotator_sexuality_gay,annotator_sexuality_straight,annotator_sexuality_other
0,47777,10873,3,0.0,0.0,0.0,0.0,2.0,0.0,0.0,...,False,False,False,False,False,False,False,False,True,False
1,39773,2790,2,0.0,0.0,0.0,0.0,2.0,0.0,0.0,...,False,False,False,False,False,False,False,False,True,False
2,47101,3379,3,4.0,4.0,4.0,4.0,4.0,4.0,0.0,...,False,False,False,False,True,False,False,False,True,False
3,43625,7365,3,2.0,3.0,2.0,1.0,2.0,0.0,0.0,...,False,False,False,False,False,False,False,False,True,False
4,12538,488,0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,...,False,False,False,False,False,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135551,37080,8590,2,1.0,1.0,0.0,0.0,2.0,0.0,0.0,...,False,False,False,False,False,False,False,False,True,False
135552,22986,8303,2,2.0,0.0,0.0,0.0,2.0,0.0,0.0,...,False,False,False,False,False,True,True,False,False,False
135553,21008,6207,2,1.0,1.0,1.0,1.0,1.0,0.0,0.0,...,False,False,False,False,False,False,False,False,True,False
135554,22986,7886,2,2.0,0.0,0.0,0.0,2.0,0.0,0.0,...,False,False,False,False,True,False,False,False,True,False


### Creating a new dataset 

#### Check if same hate_speech_score for same comment_id 

In [5]:
def check_score_for_same_comments(df):
    """
    Check if the values of hate_speech_score are the same for a same comment_id. 
    The function return all the id_comment where it is not the same.
    """
    inconsistent_rows = []
    all_id_comment = np.unique(df["comment_id"])
    for id_comment in all_id_comment:
        list_score_hate_speech = np.array(df.loc[df["comment_id"]==id_comment]["hate_speech_score"])
        
        if np.all(list_score_hate_speech != list_score_hate_speech[0]) : 
            inconsistent_rows.append(id_comment)
            
    return inconsistent_rows

In [6]:
check_score_for_same_comments(df)

[]

#### Liste des colonnes 

In [6]:
columns = df.columns.values.tolist()
columns

['comment_id',
 'annotator_id',
 'platform',
 'sentiment',
 'respect',
 'insult',
 'humiliate',
 'status',
 'dehumanize',
 'violence',
 'genocide',
 'attack_defend',
 'hatespeech',
 'hate_speech_score',
 'text',
 'infitms',
 'outfitms',
 'annotator_severity',
 'std_err',
 'annotator_infitms',
 'annotator_outfitms',
 'hypothesis',
 'target_race_asian',
 'target_race_black',
 'target_race_latinx',
 'target_race_middle_eastern',
 'target_race_native_american',
 'target_race_pacific_islander',
 'target_race_white',
 'target_race_other',
 'target_race',
 'target_religion_atheist',
 'target_religion_buddhist',
 'target_religion_christian',
 'target_religion_hindu',
 'target_religion_jewish',
 'target_religion_mormon',
 'target_religion_muslim',
 'target_religion_other',
 'target_religion',
 'target_origin_immigrant',
 'target_origin_migrant_worker',
 'target_origin_specific_country',
 'target_origin_undocumented',
 'target_origin_other',
 'target_origin',
 'target_gender_men',
 'target_gende

In [128]:
def interesting_columns_for_classification(df):
    """
    The function returns  the target columns that fall into one of these categories : "gender", "race", "religion", "origin","sexuality","age","disability". Note that in the dataset, there is no profession target.
    We will consider that all and only target_race_ falls into race, all and only target_religion_ falls into religion, all and only target_gender falls into gender and same for others.
    """
    all_columns = df.columns.values.tolist()
    
    interesting_columns_target_race = []
    interesting_columns_target_religion = []
    interesting_columns_target_gender = []
    interesting_columns_target_origin=[]
    interesting_columns_target_sexuality=[]
    interesting_columns_target_age=[]
    interesting_columns_target_disability=[]
    
    for name_column in all_columns:
        if 'target_race' in name_column :
            interesting_columns_target_race.append(name_column) 
        if 'target_religion' in name_column:
            interesting_columns_target_religion.append(name_column)
        if 'target_gender' in name_column:
            interesting_columns_target_gender.append(name_column)
        if 'target_origin' in name_column:
            interesting_columns_target_origin.append(name_column)
        if 'target_sexuality' in name_column:
            interesting_columns_target_sexuality.append(name_column)
        if 'target_age' in name_column:
            interesting_columns_target_age.append(name_column)
        if 'target_disability' in name_column:
            interesting_columns_target_disability.append(name_column)
            
    return interesting_columns_target_race,interesting_columns_target_gender,interesting_columns_target_religion,interesting_columns_target_origin,interesting_columns_target_sexuality,interesting_columns_target_age,interesting_columns_target_disability

#### Create new dataset 

In [144]:
def assign_bias_for_each_row(df,interesting_columns_target_race,interesting_columns_target_gender,interesting_columns_target_religion,interesting_columns_target_origin,interesting_columns_target_sexuality,interesting_columns_target_age,interesting_columns_target_disability):
    """
    The goal is to classify one comment into one of the following bias : gender, race, profession, religion. 
    The function takes as parameters a small dataframe where all the id_comments and text are the same, and the columns in the original dataset that we are interested to make this classification. Note that there is no profession target in this dataset. 
    
    We calcule the number of times each target is True for race, gender, religion, origin, age, sexuality or disability.
    If we can observe that there is a higner number of times True for origin, age, sexuality or disability than for race, gender, religion, then we skip this comment.
    Else,   among race, gender, religion,  we keep only the target that has the highest number and then, the comment will be classified into the bias that corresponds to that target.
            In the extreme case, if two targets have the same number or three tragets have the same number, we pick one of the two randomly. 
    """
    
    
    #This corresponds to how to calculate the number of time each target is True
    #there is no interesting columns for profession
    df = df.astype(int)
    df = df.sum().to_frame().T
    target_race = df[interesting_columns_target_race].sum(axis=1)[0] #number of times the target race has been equal to true
    target_gender = df[interesting_columns_target_gender].sum(axis=1)[0] #number of times the target gender has been equal to true
    target_religion = df[interesting_columns_target_religion].sum(axis=1)[0] #number of times the target religion has been equal to true
    target_origin = df[interesting_columns_target_origin].sum(axis=1)[0] #number of times the target origin has been equal to true
    target_sexuality = df[interesting_columns_target_sexuality].sum(axis=1)[0] #number of times the target sexuality has been equal to true
    target_age = df[interesting_columns_target_age].sum(axis=1)[0] #number of times the target age has been equal to true
    target_disability = df[interesting_columns_target_disability].sum(axis=1)[0] #number of times the target disability has been equal to true

    if target_race == target_gender == target_religion == 0: #if the three targets have never been equal to true, then we return None and we will skip this comment 
        return None
    
    index_target_max = np.argmax([target_race,target_gender,target_religion,target_origin,target_sexuality,target_age,target_disability]) #which target has been equal to true the most frequently
    if index_target_max == 0:
        target = "race"
        target_value_max = target_race
    elif index_target_max == 1:
        target = "gender"
        target_value_max = target_gender
    elif index_target_max == 2:
        target = "religion"
        target_value_max = target_religion
    else:
        return None #this is the case when one of these : target_origin, target_age, target_sexuality, target_disability has a higher value than one of these : target_gender, target_race, target_religion
    
    random.seed(42)
    if target_race == target_gender == target_religion: #if the three targets have the same number, then we choose one of them randomly
        i = random.randrange(3) #so we choose one target randomly
        if i==0:
            target = "race"
        elif i==1:
            target = "gender"
        else:
            target = "religion"
            
    else:
        if target_race == target_gender == target_value_max: #if two targets have the same number, and this number is the highest one, then we choose randomly one of the two.
            i = random.randrange(2) #so we choose one of the two targets randomly
            if i==0:
                target = "race"
            else:
                target = "gender"
        elif target_race == target_religion == target_value_max: #if two targets have the same number, and this number is the highest one, then we choose randomly one of the two.
            i = random.randrange(2) #so we choose one of the two targets randomly
            if i==0:
                target = "race"
            else:
                target = "religion"
        elif target_gender == target_religion == target_value_max: #if two targets have the same number, and this number is the highest one, then we choose randomly one of the two.
            i = random.randrange(2) #so we choose one of the two targets randomly
            if i==0:
                target = "gender"
            else:
                target = "religion"
    
    return target

In [145]:
def create_new_dataframe(df):
    """
    We create a new dataframe where the columns are id_comment, text, gender, race, religion, profession, hateful, non-hateful, unrelated.
    Each id_comment is unique.
    
    How to choose in which bias the comment falls into ?
    
    we call the function assign_bias_for_each_row. 
    
    How to choose wheither the comment is hateful ornon-hateful  ?
    
    We consider that the hate_speech_score is the same for all comments in the original dataset that have the same id_comment.
    For one comment, we extract the hate_speech_score. 
    If this score is strictly higher than -0.8, it is classified as hateful. 
    If this score is  lower than -0.8, it is classified as non-hateful. 
       
    The function returns the new dataset .
    """
    
    
    interesting_columns_target_race,interesting_columns_target_gender,interesting_columns_target_religion,interesting_columns_target_origin,interesting_columns_target_sexuality,interesting_columns_target_age,interesting_columns_target_disability= interesting_columns_for_classification(df)
    interesting_columns = interesting_columns_target_race + interesting_columns_target_religion + interesting_columns_target_gender + interesting_columns_target_origin + interesting_columns_target_sexuality + interesting_columns_target_age + interesting_columns_target_disability
    
    
    all_id_comment = np.unique(df["comment_id"])
    
    
    new_df = pd.DataFrame(columns=["comment_id","text","race","gender","profession","religion","hateful","non-hateful"])
    
    for id_comment in all_id_comment:
        
        text = df.loc[df["comment_id"]==id_comment]["text"].iloc[0]
        hate_speech_score = df.loc[df["comment_id"]==id_comment]["hate_speech_score"].iloc[0]
        target = assign_bias_for_each_row(df.loc[df["comment_id"]==id_comment][interesting_columns],interesting_columns_target_race,interesting_columns_target_gender,interesting_columns_target_religion,interesting_columns_target_origin,interesting_columns_target_sexuality,interesting_columns_target_age,interesting_columns_target_disability)
        
        if target != None:
            
            new_row = pd.DataFrame({'comment_id':[id_comment],'text':[text], 'race':[target=='race'], 'gender':[target=='gender'], 'profession':[target=='profession'], 'religion':[target=='religion'], 'hateful': [hate_speech_score>-0.8], 'non-hateful': [hate_speech_score<=-0.8]})
            new_df = pd.concat([new_df, new_row], ignore_index=True)
        
    return new_df

In [146]:
new_df = create_new_dataframe(df)
new_df

,comment_id,text,race,gender,profession,religion,hateful,non-hateful
0,1,White people are trash.,True,False,False,False,True,False
1,3,"I'm not saying you said that, I'm just saying ...",True,False,False,False,False,True
2,5,Fabrice Fabrice is ostensibly black or black/l...,True,False,False,False,False,True
3,6,First off you look cool as fuck! Anyway if we ...,False,True,False,False,True,False
4,8,Guranteed at least one of these guys has raped...,False,True,False,False,True,False
...,...,...,...,...,...,...,...,...
27839,50062,So smart. They got the most Christian looking ...,False,False,False,True,False,True
27840,50065,With all this racism still alive and well in 2...,True,False,False,False,True,False
27841,50068,That Kenyan girl is gorgeous!,False,True,False,False,False,True
27842,50069,Yikes!!! Bad enough she'shard to look at.every...,False,True,False,False,True,False


#### Add a label 

In [149]:
def add_labels_column(df):
    """
    The function adds a column column labels to the df.
    """
    labels = []
    for idx in df.index: #iterating over the df
        
        d = df.loc[idx].to_dict() #transform the corresponding row into a dict
        bias=""
        how_hateful=""
        for (key,value) in d.items():
            if value == True: 
                if key == 'hateful' or key =='non-hateful':
                    how_hateful = key
                else:
                    bias = key
        labels.append(bias+"_"+how_hateful)
    
    df["labels"] = labels
    
    return df

In [151]:
new_df = add_labels_column(new_df)

### Cleaning the comments 

In [17]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/rubenhayat/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [152]:
sw = stopwords.words('english')

In [153]:
def clean_text(text):
    text=text.lower() #On met tout en minuscule
    text = re.sub(r"[^a-zA-Z?.!,¿]+", " ", text) # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")
    text = re.sub(r"http\S+", "",text) #Removing URLs
    #text = html.sub(r'',text) #Removing html tags
    
    punctuations = '@#!?+&*[]-%.:/();$=><|{}^' + "'`" + '_'
    for p in punctuations:
        text = text.replace(p,'') #Removing punctuations
        
    text = [word.lower() for word in text.split() if word.lower() not in sw]
    text = " ".join(text) #removing stopwords
    
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    
    text = emoji_pattern.sub(r'', text) #Removing emojis
    return text

In [154]:
new_df['text'] = new_df['text'].apply(lambda x: clean_text(x))

### Tokenization and Embedding 

In [37]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

#### Example for 1st comment 

In [155]:
new_df['text'].iloc[0]

'white people trash'

In [156]:
tokenizer.tokenize(new_df['text'].iloc[0])

['white', 'people', 'trash']

In [157]:
tokenizer.convert_tokens_to_ids(tokenizer.tokenize(new_df['text'].iloc[0]))

[2317, 2111, 11669]

#### defining max_len 

In [158]:
def max_len(df):
    max_len = 0
    for comment in df['text']:
        inputs_id = tokenizer.encode(comment,add_special_tokens=True)
        
        max_len = max(max_len, len(inputs_id))
    return max_len

In [159]:
max_len = max_len(new_df)

In [160]:
inputs_id = []
attention_masks = []

for comment in new_df['text']:
    encoded_dict = tokenizer.encode_plus(
                        comment,
                        add_special_tokens = True, #Add CLS and SEP
                        max_length = max_len, 
                        padding = 'max_length',
                        return_tensors = 'pt' #return pytorch tensors
                    )
    inputs_id.append(encoded_dict['input_ids'])
    attention_masks.append(encoded_dict['attention_mask'])
    
# Convert the lists into tensors.
input_ids = torch.cat(inputs_id, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
print('Token IDs:', input_ids[0])

Token IDs: tensor([  101,  2317,  2111, 11669,   102,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,    